In [23]:
# final
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

In [24]:
# Глобальная переменная для хранения содержания лекции
lecture_content = ""

def read_lecture(file_path):
    global lecture_content
    with open(file_path, 'r', encoding='utf-8') as f:
        lecture_content = f.read()

def process_lecture():
    
    global lecture_content
    threshold=1.9
    cosine_similarity_threshold=0.8
    
    # Очистка лекции от лишних символов
    lecture_clean = re.sub(r'[^\w\s.\\n\n]', '', lecture_content)
    lecture_clean = lecture_clean.replace('\n', '')

    # Замена точек на пробелы для последующего токенизирования
    lecture_clean_words = lecture_clean.replace('.', ' ')
    
    # Удаление стоп-слов и токенизация
    stopWords = set(stopwords.words("russian"))
    words = word_tokenize(lecture_clean_words)

    # Подсчет частоты слов
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    # Токенизация предложений и определение роли каждого предложения
    sentences = sent_tokenize(lecture_clean)
    sentenceRole = dict()
    
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceRole:
                    sentenceRole[sentence] += freq
                else:
                    sentenceRole[sentence] = freq
    
    sumRole = sum(sentenceRole.values())
    averageRole = int(sumRole / len(sentenceRole))

    # Использование TF-IDF для выделения ключевых предложений
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    summary_sentences = [sentences[i] for i in range(len(sentences))
                         if (sentenceRole[sentences[i]] > (threshold*averageRole) and
                             np.max(cosine_similarities[i, :]) > cosine_similarity_threshold)]

    # Формирование итогового резюме
    summary = ' '.join(summary_sentences)
    
    return summary

def extract_topics(num_topics=5):
    global lecture_content
    
    # Удаление стоп-слов и токенизация
    stop_words = stopwords.words("russian")
    vectorizer = CountVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform([lecture_content])

    # Применение LDA для извлечения тем
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=2)
    lda.fit(X)

    train_topics = lda.transform(X)

    topic_word_matrix = lda.components_

    # Извлечение топ-10 слов для каждой темы
    topics_words = []
    feature_names = vectorizer.get_feature_names_out()
    for topic_weights in topic_word_matrix:
        topic_words = [feature_names[i] for i in topic_weights.argsort()[:-10 - 1:-1]]
        topics_words.append(topic_words)

    return train_topics, topics_words

# Список файлов для лекций
lecture_files = ["lecture1.txt", "lecture2.txt", "lecture3.txt", "lecture4.txt", "lecture5.txt"]

# Обработка каждой лекции отдельно
for i, lecture_file in enumerate(lecture_files):
    read_lecture(lecture_file)
    
    # Обработка лекции и вывод результатов
    lecture_summary = process_lecture()
    print(f"Summary for {lecture_file}:\n{lecture_summary}")

    lecture_topics, topics_words = extract_topics()
    print(f"Topics for {lecture_file}: {', '.join(topics_words[0])}")
    print("\n")


Summary for lecture1.txt:
В начале 1990х годов объединение элементов ООП в Паскале с визуальной технологией программирования привело к созданию системы программирования Delphi.7.2 Структура процедурных языков программирования высокого уровняВо всяком языке программирования определены способы организации данных и способы организаций действий над данными. Следующие конструкции представляют собой комментарии и поэтому пропускаются компиляторомлюбой текст не содержащий символ фигурная скобка любой текст не содержащий символы звездочка круглая скобкапоследующий текст до конца строки.Буквы русского алфавита употребляются только в комментариях символьных и текстовых константах.Концепция типов данных является одной из центральных в любом языке программирования. Паскаль характеризуется большим разнообразием типов данных отраженным на рисунке 9.Рисунок 9  Система типов данных языка Паскаль79Тип данных называется порядковым если он состоит из счетного количества значений которые можно пронумерова